<a href="https://colab.research.google.com/github/Guliko24/NetZoo_network/blob/main/GRN_26_MCF7_vs_MDA-MB231_BrCA_TF_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import all the packages as needed
import pandas as pd
import networkx as nx

In [3]:
# prompt: let's load Gdrive onto notebook

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Navigate to your Google Drive files
%cd /content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets


/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets


In [5]:
import numpy as np
import pandas as pd

def log_scale_dataframe(df):
    """Applies log scaling transformation to a TF-gene interaction DataFrame."""
    return np.sign(df) * np.log1p(np.abs(df))

def reverse_log_scale_dataframe(scaled_df):
    """Reverses log scaling transformation for a DataFrame."""
    return np.sign(scaled_df) * (np.expm1(np.abs(scaled_df)))


def filter_by_threshold(matrix: pd.DataFrame, threshold: float, mode: str = 'greater') -> pd.DataFrame:
    """
    Filter the matrix based on a threshold.
    Parameters:
        matrix: The TF vs Genes interaction matrix.
        threshold: The threshold value for filtering.
        mode: 'greater' to keep values greater than threshold, 'less' to keep values less than threshold.
    """
    if mode == 'greater':
        return matrix[matrix > threshold].fillna(0)
    elif mode == 'less':
        return matrix[matrix < threshold].fillna(0)
    else:
        raise ValueError("Mode must be either 'greater' or 'less'")

def binarize_matrix(matrix: pd.DataFrame, threshold: float) -> pd.DataFrame:
    """
    Binarize the matrix based on a threshold.
    Values greater than or equal to the threshold become 1, otherwise 0.
    """
    binary_matrix = (matrix >= threshold).astype(int)
    return binary_matrix

def rank_interactions(matrix: pd.DataFrame, top_n: int = 5) -> pd.DataFrame:
    """
    Rank the top N interactions for each transcription factor.
    Parameters:
        matrix: The TF vs Genes interaction matrix.
        top_n: Number of top interactions to return for each TF.
    """
    ranked_interactions = pd.DataFrame()
    for tf in matrix.index:
        top_genes = matrix.loc[tf].nlargest(top_n)
        ranked_interactions = pd.concat([ranked_interactions, top_genes], axis=1)
    return ranked_interactions.T

def aggregate_interactions(matrix: pd.DataFrame, axis: int = 0) -> pd.Series:
    """
    Aggregate interaction strengths.
    Parameters:
        axis: 0 to aggregate across genes (per TF), 1 to aggregate across TFs (per Gene).
    """
    return matrix.sum(axis=axis)

def construct_interaction_network(matrix: pd.DataFrame, threshold: float) -> pd.DataFrame:
    """
    Construct an interaction network by keeping only interactions above a threshold.
    Returns a DataFrame representing edges in the network.
    """
    filtered_matrix = filter_by_threshold(matrix, threshold, mode='greater')
    edges = []
    for tf in filtered_matrix.index:
        for gene in filtered_matrix.columns:
            if filtered_matrix.loc[tf, gene] > 0:
                edges.append((tf, gene, filtered_matrix.loc[tf, gene]))
    network_df = pd.DataFrame(edges, columns=['TF', 'Gene', 'Interaction_Strength'])
    return network_df

def split_positive_negative_matrices(matrix: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    Split the matrix into two DataFrames: one for positive values and one for negative values.
    Ensure that the matrix values are numeric by coercing errors and replacing non-numeric values with NaN.
    Retain the index and columns in both matrices.
    """
    matrix_numeric = matrix.apply(pd.to_numeric, errors='coerce')
    positive_matrix = matrix_numeric.where(matrix_numeric > 0).fillna(0)
    negative_matrix = matrix_numeric.where(matrix_numeric < 0).fillna(0)
    positive_matrix.index = matrix.index
    positive_matrix.columns = matrix.columns
    negative_matrix.index = matrix.index
    negative_matrix.columns = matrix.columns
    return positive_matrix, negative_matrix

def check_repeated_indices_columns(matrix: pd.DataFrame):
    """
    Check for repeated transcription factors (rows) and genes (columns) in the matrix.
    """
    repeated_rows = matrix.index[matrix.index.duplicated()].unique()
    repeated_columns = matrix.columns[matrix.columns.duplicated()].unique()

    print("Repeated Rows (TFs):")
    if len(repeated_rows) > 0:
        print(repeated_rows)
    else:
        print("No repeated rows found.")

    print("\nRepeated Columns (Genes):")
    if len(repeated_columns) > 0:
        print(repeated_columns)
    else:
        print("No repeated columns found.")



# **1. Let's process the gene expression data set GSE73857 dataset for MCF7 and MDA-MB231 cells**

*the* oridinal *gene* expression count file had ENSEMBL id for genes instead of their names, so I use R studio and Biomart package to find the gene names and add as an additional column, however it seems that there are unannotated genes which we need to remove later on.
the data set also had experimental duplicates for MCF7 and triplicates for MDA-MB231 cells so I averaged them. I could have done student-T test and find significantly different genes, but I went ahead with averaging and calculating fold-change.

In [6]:
import pandas as pd

# Load RNA-seq data
df = pd.read_csv("/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/GSE73857_MCF7_MDA_gene_expression.txt", sep="\t")  # Replace with your RNA-seq file



# Step 1: Remove the `ensembl.id` column
if "ensembl.id" in df.columns:
    df.drop(columns=["ensembl.id"], inplace=True)

# Step 2: Move the `gene_name` column to be the first column
if "gene_name" in df.columns:
    gene_name_column = df.pop("gene_name")  # Remove and retrieve `gene.name` column
    df.insert(0, "gene_name", gene_name_column)  # Insert it as the first column

# Step 3: Set `gene_name` as the index
df.set_index("gene_name", inplace=True)


# Step 1: Calculate averages
MCF7_average = df.iloc[:, [0, 1]].mean(axis=1)
MDA_MB231_average = df.iloc[:, [2, 3, 4]].mean(axis=1)

# Step 2: Drop the original columns
df.drop(df.columns[[0, 1, 2, 3, 4]], axis=1, inplace=True)

# Step 3: Add the new averaged columns
df["MCF7"] = MCF7_average
df["MDA-MB231"] = MDA_MB231_average

# Step 1: Remove rows where both `MCF7` and `MDA-MB231` have zero expression
MCF7_MDA_filtered_expression = df[(df["MCF7"] != 0) | (df["MDA-MB231"] != 0)]



In [7]:
MCF7_MDA_filtered_expression.head()

,MCF7,MDA-MB231
gene_name,,
TSPAN6,494.000,1198.373333
DPM1,4458.000,2367.000000
SCYL3,851.405,406.156667
FIRRM,1408.270,1153.816667
FGR,0.000,0.333333


In [8]:
MCF7_MDA_filtered_expression.info()


<class 'pandas.core.frame.DataFrame'>
Index: 40677 entries, TSPAN6 to nan
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MCF7       40677 non-null  float64
 1   MDA-MB231  40677 non-null  float64
dtypes: float64(2)
memory usage: 953.4+ KB


# **2. Lets check whether there are differently expressed genes between two cell line so I could narrow down my graphs, by checking the fold change:**

In [9]:
import pandas as pd
import numpy as np



# Step 1: Separate rows with zero expression for further investigation
zero_expression_genes = MCF7_MDA_filtered_expression[
    (MCF7_MDA_filtered_expression['MCF7'] == 0) | (MCF7_MDA_filtered_expression['MDA-MB231'] == 0)
].copy()

# Save genes with zero expression to a new file
zero_expression_genes.to_csv("zero_expression_genes.tsv", sep="\t", index=False)

# Step 2: Filter out rows with zero expression from the original DataFrame
MCF7_MDA_filtered_expression_no_zeros = MCF7_MDA_filtered_expression[
    (MCF7_MDA_filtered_expression['MCF7'] != 0) & (MCF7_MDA_filtered_expression['MDA-MB231'] != 0)
].copy()

# Step 3: Calculate fold change and log2 fold change
MCF7_MDA_filtered_expression_no_zeros['fold_change'] =MCF7_MDA_filtered_expression_no_zeros['MDA-MB231'] / MCF7_MDA_filtered_expression_no_zeros['MCF7']
MCF7_MDA_filtered_expression_no_zeros.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace inf with NaN
MCF7_MDA_filtered_expression_no_zeros.dropna(subset=['fold_change'], inplace=True)  # Drop rows with NaN in fold_change
MCF7_MDA_filtered_expression_no_zeros['log2_fold_change'] = np.log2(MCF7_MDA_filtered_expression_no_zeros['fold_change'])

# Step 4: Filter for significant fold change (e.g., >2 or <0.5)
significant_genes = MCF7_MDA_filtered_expression_no_zeros[
    (MCF7_MDA_filtered_expression_no_zeros['fold_change'] > 2) | (MCF7_MDA_filtered_expression_no_zeros['fold_change'] < 0.5)
]

# Save the cleaned DataFrame and significant genes to new files
MCF7_MDA_filtered_expression_no_zeros.to_csv("cleaned_gene_expression.tsv", sep="\t", index=False)
significant_genes.to_csv("significant_genes.tsv", sep="\t", index=False)

print("Files generated:")
print("1. 'zero_expression_genes.tsv' for further investigation.") #####this is an important file as it may show which genes switched on/off
print("2. 'cleaned_gene_expression.tsv' with cleaned data.")#####this is a file which has zero expression genes removed
print("3. 'significant_genes.tsv' with significant fold change.")####this is a file which as 2 fold change and 0.5 fold chane between 2 cell lines


Files generated:
1. 'zero_expression_genes.tsv' for further investigation.
2. 'cleaned_gene_expression.tsv' with cleaned data.
3. 'significant_genes.tsv' with significant fold change.


In [10]:
significant_genes.head()

,MCF7,MDA-MB231,fold_change,log2_fold_change
gene_name,,,,
TSPAN6,494.000,1198.373333,2.425857,1.278494
SCYL3,851.405,406.156667,0.477043,-1.067809
CFH,1.000,840.653333,840.653333,9.715367
STPG1,119.010,406.240000,3.413495,1.771249
NIPAL3,563.490,2629.426667,4.666324,2.222286


In [11]:
# Remove rows with NaN in the index (gene names)
# Remove rows with NaN in the index (if gene names are in the index)
significant_genes_cleaned = significant_genes[~significant_genes.index.isnull()]

# Save the cleaned significant genes to a new file
significant_genes_cleaned.to_csv("significant_genes_cleaned.tsv", sep="\t")

print("Significant genes with NaN in gene names removed. Saved to 'significant_genes_cleaned.tsv'.")


Significant genes with NaN in gene names removed. Saved to 'significant_genes_cleaned.tsv'.


In [ ]:
significant_genes_cleaned.info()

# **3. Let's modify our TF-Gene matrices per cell line keeping only the genes found in significant_genes_cleaned**

In [12]:
#let's read the MCF7_TF_Genes dataset and make changes
df_MCF7_TF_Genes = pd.read_csv('/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/ACH-000019_TF_vs_Genes_total expression.csv', index_col=0)
df_MDA_MB231_TF_Genes=pd.read_csv('/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/ACH-000768.csv', index_col=0)
#df_gene_expression=pd.read_csv('/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/CCLE_expression_withinSample.csv', index_col=0)
#

In [13]:

# Step 1: Load the TF-genes matrix and the filtered gene list
#df_MCF7_TF_Genes ===matrix
#df_MDA_MB231_TF_Genes ===matrix
#MCF7_MDA_filtered_expression===gene expression file



# Extract the gene names from df_filtered
filtered_genes = set(significant_genes_cleaned.index)  # Assuming `gene_name` is the column with gene names

# Step 3: Filter the columns of the TF-genes matrix
MCF7_filtered_tf_gene_matrix = df_MCF7_TF_Genes.loc[:, df_MCF7_TF_Genes.columns.isin(filtered_genes)]
MDA_MB231_filtered_tf_gene_matrix = df_MDA_MB231_TF_Genes.loc[:, df_MDA_MB231_TF_Genes.columns.isin(filtered_genes)]



In [14]:
MCF7_filtered_tf_gene_matrix.info()
MDA_MB231_filtered_tf_gene_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1132 entries, AHR to ZSCAN9
Columns: 6445 entries, A1BG to TMEM92
dtypes: float64(6445)
memory usage: 55.7+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 1132 entries, AHR to ZSCAN9
Columns: 7229 entries, A1BG to ZZZ3
dtypes: float64(7229)
memory usage: 62.4+ MB


# **4. Lets approach this slightly differently; lets rank genes based on log2_fold_change in the significant_genes_cleaned file. Then select top 300 upregulated genes and top 300 downregulated genes and only then use these for graph construction**

In [15]:
import pandas as pd


# Sort genes by log2 fold change (descending)
ranked_genes = significant_genes_cleaned.sort_values(
    by="log2_fold_change", ascending=False
)

# Save the ranked genes to a new file
ranked_genes.to_csv("ranked_significant_genes_cleaned.csv", index=False)

# Display top upregulated and downregulated genes
top_upregulated = ranked_genes.head(300)  # Top 300 overexpressed genes
top_downregulated = ranked_genes.tail(300)  # Top 300 downexpressed genes

print("Top 300 Upregulated Genes:")
print(top_upregulated)

print("\nTop 300 Downregulated Genes:")
print(top_downregulated)


Top 300 Upregulated Genes:
           MCF7    MDA-MB231  fold_change  log2_fold_change
gene_name                                                  
TNFRSF10D   0.5  2168.643333  4337.286667         12.082577
CD40        0.5  1843.333333  3686.666667         11.848101
ATP10D      0.5  1417.743333  2835.486667         11.469381
STAC        0.5  1268.000000  2536.000000         11.308339
TCEAL9      1.0  2219.333333  2219.333333         11.115911
...         ...          ...          ...               ...
GCNT2      27.5  4943.076667   179.748242          7.489834
APOL1       4.5   806.653333   179.256296          7.485880
MCAM       21.0  3759.760000   179.036190          7.484107
KLRC2       0.5    89.486667   178.973333          7.483601
ME3        12.0  2131.666667   177.638889          7.472804

[300 rows x 4 columns]

Top 300 Downregulated Genes:
               MCF7  MDA-MB231  fold_change  log2_fold_change
gene_name                                                    
HUNK        203

# **5. Filter the TF-Gene matrix to contain top 300 upregulated and top 300 downregulated genes per cell line**

In [16]:
# Ensure gene names are not used as index
if top_upregulated.index.name == 'gene_name':
    top_upregulated.reset_index(inplace=True)
if top_downregulated.index.name == 'gene_name':
    top_downregulated.reset_index(inplace=True)

In [17]:
# Combine the two lists
combined_genes_upregulated_downregulated = pd.concat([top_upregulated, top_downregulated], ignore_index=True)

combined_genes_upregulated_downregulated.head()

,gene_name,MCF7,MDA-MB231,fold_change,log2_fold_change
0,TNFRSF10D,0.5,2168.643333,4337.286667,12.082577
1,CD40,0.5,1843.333333,3686.666667,11.848101
2,ATP10D,0.5,1417.743333,2835.486667,11.469381
3,STAC,0.5,1268.000000,2536.000000,11.308339
4,TCEAL9,1.0,2219.333333,2219.333333,11.115911


In [18]:
combined_genes_upregulated_downregulated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gene_name         600 non-null    object 
 1   MCF7              600 non-null    float64
 2   MDA-MB231         600 non-null    float64
 3   fold_change       600 non-null    float64
 4   log2_fold_change  600 non-null    float64
dtypes: float64(4), object(1)
memory usage: 23.6+ KB


# The files we generate below may be important in the future if we want to split positive and negative graphs per cell line again

In [ ]:
import pandas as pd
#df_MCF7_TF_Genes ===matrix
#df_MDA_MB231_TF_Genes ===matrix
#top_upregulated = ranked_genes.head(300)  # Top 300 overexpressed genes
#top_downregulated = ranked_genes.tail(300)  # Top 300 downexpressed genes



# Filter the TF-Gene matrix for upregulated genes
#MCF7_upregulated_matrix = df_MCF7_TF_Genes.loc[:, df_MCF7_TF_Genes.columns.isin(top_upregulated)]
#MDA_MB231_upregulated_matrix = df_MDA_MB231_TF_Genes.loc[:, df_MDA_MB231_TF_Genes.columns.isin(top_upregulated)]

# Filter the TF-Gene matrix for downregulated genes
#MCF7_downregulated_matrix = df_MCF7_TF_Genes.loc[:, df_MCF7_TF_Genes.columns.isin(top_downregulated)]
#MDA_MB231_downregulated_matrix = df_MDA_MB231_TF_Genes.loc[:, df_MDA_MB231_TF_Genes.columns.isin(top_downregulated)]


# Save the filtered matrices
#MCF7_upregulated_matrix.to_csv("MCF7_upregulated_tf_gene_matrix.tsv", sep="\t")
#MCF7_downregulated_matrix.to_csv("MCF7_downregulated_tf_gene_matrix.tsv", sep="\t")
#MDA_MB231_upregulated_matrix.to_csv("MDA_MB231_upregulated_tf_gene_matrix.tsv", sep="\t")
#MDA_MB231_downregulated_matrix.to_csv("MDA_MB231_downregulated_tf_gene_matrix.tsv", sep="\t")

#print("Filtered matrices saved.")


# **BUILDING GRAPHS**:
-> lets combine top 300 upregulated and top 300 downregulated genes

-> filter the Tf-gene matrix based on these genes per cell line (MCf7 and MDA_MB231)

-> build the graphs with NetworkX and visualize

-> Do GSP with PyGSP

# **1) Combine gene list and fiter the matrix**

In [19]:
import pandas as pd



# combined gene list is 'combined_genes_upregulated_downregulated'
# Filter the matrices using the combined gene list
selected_genes = combined_genes_upregulated_downregulated['gene_name'].tolist()


# Filter the matrices to keep only the selected genes as columns
filtered_matrix_mcf7_combined_edges = df_MCF7_TF_Genes.loc[:, df_MCF7_TF_Genes.columns.isin(selected_genes)]
filtered_matrix_mda_combined_edges= df_MDA_MB231_TF_Genes.loc[:, df_MDA_MB231_TF_Genes.columns.isin(selected_genes)]

# Save the filtered matrices for reference
filtered_matrix_mcf7_combined_edges.to_csv("filtered_matrix_mcf7_combined_edges.tsv", sep="\t")
filtered_matrix_mda_combined_edges.to_csv("filtered_matrix_mda_combined_edges.tsv", sep="\t")

print("Filtered matrices saved.")


Filtered matrices saved.


In [20]:
filtered_matrix_mcf7_combined_edges.head()

,ABCA12,ACKR3,ACOT4,ACSL5,ADAMTS1,ADAMTS12,ADAMTS16,ADAMTS19,ADCY1,ADCY5,...,TLR4,TM4SF1,TM4SF18,TMEM125,TMEM156,TMEM178B,TMEM26,TMEM30B,TMEM71,TMEM92
Row,,,,,,,,,,,,,,,,,,,,,
AHR,-1.089553,-0.795315,0.219303,-0.853007,0.399470,-1.630741,0.290181,-1.703381,-1.947731,-2.183788,...,-1.020596,-1.899293,-1.735155,-0.464616,-0.088311,0.546081,-1.327963,-1.899040,-1.944349,-0.818321
AHRR,-0.986674,-0.700822,0.392719,-0.701782,0.527235,-1.661367,0.493583,-1.492458,-1.692178,-1.953388,...,-1.257743,-1.675511,-1.762027,-0.336651,-0.162217,0.729568,-1.273587,-1.976856,-1.943266,-0.680559
AIRE,0.265959,-1.189028,-0.789297,1.062564,-1.320127,2.068158,-1.493547,0.400128,0.110629,1.216715,...,-0.363425,-1.296714,0.111233,-0.754813,0.276227,-0.464796,2.755469,0.743904,0.603933,-0.703877
ALX1,0.388290,-1.544013,-1.692139,-1.567756,-1.144992,0.850550,-1.189983,-1.292651,-1.751564,-1.385865,...,1.290326,-0.197852,1.397905,-2.600899,2.596896,-1.944171,1.328845,2.033929,-0.070829,-2.532883
ALX3,-1.169286,-1.528565,-1.913900,-1.577865,-1.383676,0.547645,-0.422447,-1.874142,-2.057815,-2.036447,...,1.891228,-1.729117,0.000001,-1.674775,0.283685,-1.568709,2.271090,1.326684,-0.644946,-2.533361


In [21]:
filtered_matrix_mcf7_combined_edges.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1132 entries, AHR to ZSCAN9
Columns: 483 entries, ABCA12 to TMEM92
dtypes: float64(483)
memory usage: 4.2+ MB


In [22]:
filtered_matrix_mda_combined_edges.head()

,ABCA12,ACKR3,ACOT4,ACSL5,ADAMTS1,ADAMTS12,ADAMTS16,ADAMTS19,ADCY1,ADCY5,...,ZNF501,ZNF502,ZNF518B,ZNF521,ZNF577,ZNF630,ZNF655,ZNF71,ZNF711,ZNF732
Row,,,,,,,,,,,,,,,,,,,,,
AHR,0.304008,-0.667807,1.200416,-0.936033,0.182179,-1.235324,-1.395829,-0.644775,-2.041014,-1.388207,...,-2.349634,-2.064601,-1.247681,-1.421086,-0.907489,-1.347365,-1.693050,-1.362386,-2.278187,0.146711
AHRR,0.247151,-0.587696,1.293375,-0.796551,0.328368,-1.308294,-1.069268,-0.484905,-1.789929,-1.243058,...,-2.141010,-2.002920,-1.431786,-1.178416,-0.893068,-1.137283,-1.451020,-1.081178,-2.225300,0.240623
AIRE,0.874790,-1.013267,0.275668,0.100357,-1.201515,2.453961,-2.109349,-0.805395,-0.436276,1.693876,...,-1.956179,1.233329,0.527247,-0.029944,0.523460,-1.969279,-2.275027,-2.057922,-0.475194,1.937781
ALX1,1.901858,-1.362818,-0.506440,-1.288876,-1.512983,1.082257,-2.706565,-0.351387,-1.688240,-0.205509,...,-3.467904,-0.405839,0.920343,-1.910685,-0.323486,-0.789209,-0.803448,-2.291393,1.893416,-0.187331
ALX3,0.740310,-1.202866,-0.330167,-1.513954,-1.692206,0.859158,-2.328144,-1.254828,-2.147329,-0.570023,...,-3.229053,0.866085,0.700537,-1.929038,-0.656320,-1.514953,-2.219323,-2.912750,2.325852,0.511644


In [23]:
filtered_matrix_mda_combined_edges.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1132 entries, AHR to ZSCAN9
Columns: 535 entries, ABCA12 to ZNF732
dtypes: float64(535)
memory usage: 4.6+ MB


# We still have 1132 entries for TFs, let's trim this down based on the article we found at https://www.academia.edu/62149166/Identification_of_novel_dysregulated_key_genes_in_Breast_cancer_through_high_throughput_ChIP_Seq_data_analysis .
# I have generated a csv file with 20 TF from this article and added CTCF to the list as I have an interest in it

In [24]:
import pandas as pd
import os

# File paths
tf_file = "/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/Breast_Cancer_TFs.csv"
ppi_file = "/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/ppi_complete.txt"
output_file = "filtered_ppi.tsv"

# Validate file paths
if not os.path.exists(tf_file):
    raise FileNotFoundError(f"TF file not found: {tf_file}")
if not os.path.exists(ppi_file):
    raise FileNotFoundError(f"PPI file not found: {ppi_file}")

# Load the TF file (CSV format)
print(f"Loading TF file: {tf_file}")
tf_df = pd.read_csv(tf_file)

# Check if the expected column exists
if 'Gene/Transcription Factor' not in tf_df.columns:
    raise KeyError("Column 'Gene/Transcription Factor' not found in TF file.")

# Get the list of transcription factors
tf_list = tf_df['Gene/Transcription Factor'].str.strip().str.upper().tolist()

# Load the PPI file (TXT format)
print(f"Loading PPI file: {ppi_file}")
ppi_df = pd.read_csv(ppi_file, sep="\t", header=None, names=["Protein1", "Protein2", "InteractionStrength"])

# Ensure the expected columns are present
if not all(col in ppi_df.columns for col in ["Protein1", "Protein2", "InteractionStrength"]):
    raise KeyError("Columns 'Protein1', 'Protein2', or 'InteractionStrength' not found in PPI file.")

# Convert PPI proteins to uppercase for case-insensitive matching
ppi_df['Protein1'] = ppi_df['Protein1'].str.strip().str.upper()
ppi_df['Protein2'] = ppi_df['Protein2'].str.strip().str.upper()


# Filter the PPI dataframe to retain rows where Protein1  in the TF list
print("Filtering PPI data...")
filtered_ppi_df = ppi_df[
    (ppi_df['Protein1'].isin(tf_list))
]

#lets try to remove zero interactions to see how small the file will be
filtered_ppi_minus_zero_df = filtered_ppi_df[filtered_ppi_df['InteractionStrength'] != 0]

# Reset the index after filtering
filtered_ppi_minus_zero_df.reset_index(drop=True, inplace=True)
# Save the filtered PPI to a new file
#print(f"Saving filtered PPI to file: {output_file}")
filtered_ppi_df.to_csv(output_file, sep="\t", index=False, header=True)
filtered_ppi_minus_zero_df.to_csv("filtered_ppi_minus_zero.tsv", sep="\t", index=False, header=True)
print(f"Filtered PPI file saved successfully to {output_file}")


Loading TF file: /content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/Breast_Cancer_TFs.csv
Loading PPI file: /content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/MCF7_raw/ppi_complete.txt
Filtering PPI data...
Filtered PPI file saved successfully to filtered_ppi.tsv


In [25]:
tf_list

['SOX2',
 'PPRAG',
 'CREB1',
 'STAT5A',
 'NR2C2',
 'LEF1',
 'CREBBP',
 'ZBED1',
 'RXRA',
 'ATF1',
 'MAFA',
 'ZNF766',
 'NFKB1',
 'NFKB2',
 'REL',
 'CENPB',
 'NPAS4',
 'ZBTB7A',
 'PATZ1',
 'MZF1',
 'CTCF']

## Lets trim our matrices so that they contain the list of TFs from filtered_ppi_minus_zero.df file

# Lets filter matrix on 20TF and its interacting partners

# **For MCF7 and MDA**

In [28]:
import pandas as pd

# Assuming you have already loaded your TF-gene matrix and PPI data frame
# tf_gene_matrix is your TF-gene matrix
# filtered_ppi_minus_zero_df is your PPI data frame
#HERE I decided to expand a little and pull out 18TFs from both protein1 or protein2 in ppi file

# Step 1: Extract unique TFs from both Protein1 and Protein2 columns
unique_tfs_protein1 = filtered_ppi_minus_zero_df['Protein1'].unique()
unique_tfs_protein2 = filtered_ppi_minus_zero_df['Protein2'].unique()

# Step 2: Combine the two lists and remove duplicates
unique_tfs = pd.unique(np.concatenate([unique_tfs_protein1, unique_tfs_protein2]))

# Step 3: Filter the TF-gene matrix to include only rows where the index (TF names) is in unique_tfs
filtered_matrix_mcf7_combined_edges_ppi_TFs = filtered_matrix_mcf7_combined_edges[filtered_matrix_mcf7_combined_edges.index.isin(unique_tfs)]

#Step 4: Name the index column as 'TF'
filtered_matrix_mcf7_combined_edges_ppi_TFs.index.name = 'TF'

# Step 5: Reset the index to make 'TF' a regular column
filtered_matrix_mcf7_combined_edges_ppi_TFs = filtered_matrix_mcf7_combined_edges_ppi_TFs.reset_index()

# Now filtered_tf_gene_matrix contains only the TFs mentioned in either Protein1 or Protein2 of the PPI file
#print(filtered_matrix_mcf7_combined_edges_ppi_TFs)
filtered_matrix_mcf7_combined_edges_ppi_TFs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Columns: 484 entries, TF to TMEM92
dtypes: float64(483), object(1)
memory usage: 3.2+ MB


In [30]:
import pandas as pd

# Assuming you have already loaded your TF-gene matrix and PPI data frame
# tf_gene_matrix is your TF-gene matrix
# filtered_ppi_minus_zero_df is your PPI data frame
#HERE I decided to expand a little and pull out 18TFs from both protein1 or protein2 in ppi file

# Step 1: Extract unique TFs from both Protein1 and Protein2 columns
unique_tfs_protein1 = filtered_ppi_minus_zero_df['Protein1'].unique()
unique_tfs_protein2 = filtered_ppi_minus_zero_df['Protein2'].unique()

# Step 2: Combine the two lists and remove duplicates
unique_tfs = pd.unique(np.concatenate([unique_tfs_protein1, unique_tfs_protein2]))

# Step 3: Filter the TF-gene matrix to include only rows where the index (TF names) is in unique_tfs
filtered_matrix_mda_combined_edges_ppi_TFs = filtered_matrix_mda_combined_edges[filtered_matrix_mda_combined_edges.index.isin(unique_tfs)]

#Step 4: Name the index column as 'TF'
filtered_matrix_mda_combined_edges_ppi_TFs.index.name = 'TF'

# Step 5: Reset the index to make 'TF' a regular column
filtered_matrix_mda_combined_edges_ppi_TFs = filtered_matrix_mda_combined_edges_ppi_TFs.reset_index()

# Now filtered_tf_gene_matrix contains only the TFs mentioned in either Protein1 or Protein2 of the PPI file
#print(filtered_matrix_mda_combined_edges_ppi_TFs)
filtered_matrix_mda_combined_edges_ppi_TFs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Columns: 536 entries, TF to ZNF732
dtypes: float64(535), object(1)
memory usage: 3.5+ MB


In [31]:
filtered_matrix_mda_combined_edges_ppi_TFs.head()

,TF,ABCA12,ACKR3,ACOT4,ACSL5,ADAMTS1,ADAMTS12,ADAMTS16,ADAMTS19,ADCY1,...,ZNF501,ZNF502,ZNF518B,ZNF521,ZNF577,ZNF630,ZNF655,ZNF71,ZNF711,ZNF732
0,AHR,0.304008,-0.667807,1.200416,-0.936033,0.182179,-1.235324,-1.395829,-0.644775,-2.041014,...,-2.349634,-2.064601,-1.247681,-1.421086,-0.907489,-1.347365,-1.693050,-1.362386,-2.278187,0.146711
1,AHRR,0.247151,-0.587696,1.293375,-0.796551,0.328368,-1.308294,-1.069268,-0.484905,-1.789929,...,-2.141010,-2.002920,-1.431786,-1.178416,-0.893068,-1.137283,-1.451020,-1.081178,-2.225300,0.240623
2,AIRE,0.874790,-1.013267,0.275668,0.100357,-1.201515,2.453961,-2.109349,-0.805395,-0.436276,...,-1.956179,1.233329,0.527247,-0.029944,0.523460,-1.969279,-2.275027,-2.057922,-0.475194,1.937781
3,ALX1,1.901858,-1.362818,-0.506440,-1.288876,-1.512983,1.082257,-2.706565,-0.351387,-1.688240,...,-3.467904,-0.405839,0.920343,-1.910685,-0.323486,-0.789209,-0.803448,-2.291393,1.893416,-0.187331
4,ALX3,0.740310,-1.202866,-0.330167,-1.513954,-1.692206,0.859158,-2.328144,-1.254828,-2.147329,...,-3.229053,0.866085,0.700537,-1.929038,-0.656320,-1.514953,-2.219323,-2.912750,2.325852,0.511644


In [32]:
filtered_matrix_mcf7_combined_edges_ppi_TFs.head()

,TF,ABCA12,ACKR3,ACOT4,ACSL5,ADAMTS1,ADAMTS12,ADAMTS16,ADAMTS19,ADCY1,...,TLR4,TM4SF1,TM4SF18,TMEM125,TMEM156,TMEM178B,TMEM26,TMEM30B,TMEM71,TMEM92
0,AHR,-1.089553,-0.795315,0.219303,-0.853007,0.399470,-1.630741,0.290181,-1.703381,-1.947731,...,-1.020596,-1.899293,-1.735155,-0.464616,-0.088311,0.546081,-1.327963,-1.899040,-1.944349,-0.818321
1,AHRR,-0.986674,-0.700822,0.392719,-0.701782,0.527235,-1.661367,0.493583,-1.492458,-1.692178,...,-1.257743,-1.675511,-1.762027,-0.336651,-0.162217,0.729568,-1.273587,-1.976856,-1.943266,-0.680559
2,AIRE,0.265959,-1.189028,-0.789297,1.062564,-1.320127,2.068158,-1.493547,0.400128,0.110629,...,-0.363425,-1.296714,0.111233,-0.754813,0.276227,-0.464796,2.755469,0.743904,0.603933,-0.703877
3,ALX1,0.388290,-1.544013,-1.692139,-1.567756,-1.144992,0.850550,-1.189983,-1.292651,-1.751564,...,1.290326,-0.197852,1.397905,-2.600899,2.596896,-1.944171,1.328845,2.033929,-0.070829,-2.532883
4,ALX3,-1.169286,-1.528565,-1.913900,-1.577865,-1.383676,0.547645,-0.422447,-1.874142,-2.057815,...,1.891228,-1.729117,0.000001,-1.674775,0.283685,-1.568709,2.271090,1.326684,-0.644946,-2.533361


In [35]:
def log_scale_dataframe(df):
    """Applies log scaling transformation to the interaction strengths while keeping the TF column intact."""
    df_scaled = df.copy()
    df_scaled.iloc[:, 1:] = np.sign(df.iloc[:, 1:]) * np.log1p(np.abs(df.iloc[:, 1:]))  # Apply to numeric columns only
    return df_scaled

mcf7_log_scaled = log_scale_dataframe(filtered_matrix_mcf7_combined_edges_ppi_TFs)
#
#mda_log_scaled = log_scale_dataframe(filtered_matrix_mda_combined_edges_ppi_TFs)

In [36]:
mcf7_log_scaled.head()

,TF,ABCA12,ACKR3,ACOT4,ACSL5,ADAMTS1,ADAMTS12,ADAMTS16,ADAMTS19,ADCY1,...,TLR4,TM4SF1,TM4SF18,TMEM125,TMEM156,TMEM178B,TMEM26,TMEM30B,TMEM71,TMEM92
0,AHR,-0.736950,-0.585181,0.198280,-0.616810,0.336093,-0.967266,0.254782,-0.994503,-1.081036,...,-0.703392,-1.064467,-1.006188,-0.381593,-0.084627,0.435723,-0.844994,-1.064380,-1.079888,-0.597913
1,AHRR,-0.686462,-0.531112,0.331258,-0.531676,0.423459,-0.978840,0.401178,-0.913269,-0.990351,...,-0.814366,-0.984140,-1.015965,-0.290168,-0.150329,0.547872,-0.821359,-1.090868,-1.079520,-0.519126
2,AIRE,0.235830,-0.783457,-0.581823,0.723950,-0.841622,1.121077,-0.913706,0.336563,0.104927,...,-0.310000,-0.831480,0.105471,-0.562362,0.243908,-0.381716,1.323213,0.556127,0.472458,-0.532906
3,ALX1,0.328072,-0.933743,-0.990336,-0.943032,-0.763136,0.615483,-0.783894,-0.829709,-1.012169,...,0.828694,-0.180530,0.874596,-1.281183,1.280071,-1.079827,0.845373,1.109858,-0.068433,-1.262114
4,ALX3,-0.774398,-0.927652,-1.069492,-0.946961,-0.868644,0.436734,-0.352379,-1.055754,-1.117701,...,1.061681,-1.003978,0.000001,-0.983865,0.249735,-0.943404,1.185123,0.844444,-0.497708,-1.262249


# **2. Build Graphs with Annonated Edges for each cell line**

**MCF7** **GRAPH**

In [ ]:
import networkx as nx


# Create a directed graph for MCF7
G_mcf7 = nx.DiGraph()

# Check if 'TF' is the index and reset it if necessary
if 'TF' not in filtered_matrix_mcf7_combined_edges_ppi_TFs.columns:
    filtered_matrix_mcf7_combined_edges_ppi_TFs = filtered_matrix_mcf7_combined_edges_ppi_TFs.reset_index()

# Iterate over rows in the DataFrame
for _, row in filtered_matrix_mcf7_combined_edges_ppi_TFs.iterrows():
    tf = row['TF']  # Transcription factor (First column)

    # Iterate through all genes (ignoring the first column)
    for gene, weight in row.iloc[1:].items():
        if weight > 0:
            edge_type = "positive"  # Activation
        elif weight < 0:
            edge_type = "negative"  # Repression
        else:
            continue  # Skip zero values

        # Add edge to the graph
        G_mcf7.add_edge(tf, gene, weight=weight, type=edge_type)


**MDA_MB231 GRAPH**

In [ ]:
import networkx as nx


# Create a directed graph for MDA
G_mda = nx.DiGraph()

# Check if 'TF' is the index and reset it if necessary
if 'TF' not in filtered_matrix_mda_combined_edges_ppi_TFs.columns:
    filtered_matrix_mda_combined_edges_ppi_TFs = filtered_matrix_mda_combined_edges_ppi_TFs.reset_index()

# Iterate over rows in the DataFrame
for _, row in filtered_matrix_mda_combined_edges_ppi_TFs.iterrows():
    tf = row['TF']  # Transcription factor (First column)

    # Iterate through all genes (ignoring the first column)
    for gene, weight in row.iloc[1:].items():
        if weight > 0:
            edge_type = "positive"  # Activation
        elif weight < 0:
            edge_type = "negative"  # Repression
        else:
            continue  # Skip zero values

        # Add edge to the graph
        G_mda.add_edge(tf, gene, weight=weight, type=edge_type)

# **3. Save the Graphs for Further Analysis**



In [ ]:
# Save the graphs as GraphML files
nx.write_graphml(G_mcf7, "mcf7_graph_with_annotations.graphml")
nx.write_graphml(G_mda, "mda_mb231_graph_with_annotations.graphml")

print("Graphs saved with annotated edges.")


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

def visualize_graph(graph, title):
    # Define edge colors based on annotations
    edge_colors = [
        "green" if graph[u][v]['type'] == "positive" else "red"
        for u, v in graph.edges
    ]

    # Draw the graph
    pos = nx.spring_layout(graph, seed=42)  # Spring layout for better visualization
    nx.draw(
        graph, pos, with_labels=True, edge_color=edge_colors, node_size=500, font_size=10
    )

    # Add a legend for edge colors
    plt.title(title)
    plt.show()

# Load the graphs
G_mcf7 = nx.read_graphml("mcf7_graph_with_annotations.graphml")
G_mda = nx.read_graphml("mda_mb231_graph_with_annotations.graphml")

# Visualize MCF7 graph
#visualize_graph(G_mcf7, "MCF7 Graph")

# Visualize MDA-MB231 graph
#visualize_graph(G_mda, "MDA-MB231 Graph")


LETS CONTINUE WITH BASIC GRAPH ANALYSIS:

1) **Analyze** **basic** **graph** **properties**

In [ ]:
# Number of Nodes and Edges
num_nodes_mcf7 = G_mcf7.number_of_nodes()
num_edges_mcf7 = G_mcf7.number_of_edges()
print(f"Number of nodes MCF7: {num_nodes_mcf7}")
print(f"Number of edges MCF7: {num_edges_mcf7}")

num_nodes_mda = G_mda.number_of_nodes()
num_edges_mda = G_mda.number_of_edges()
print(f"Number of nodes MDA: {num_nodes_mda}")
print(f"Number of edges MDA: {num_edges_mda}")

In [ ]:
# Degree distribution
#calculate the in-degree, out-degree, and total degree of nodes
in_degrees_mcf7 = dict(G_mcf7.in_degree())
out_degrees_mcf7 = dict(G_mcf7.out_degree())
total_degrees_mcf7 = dict(G_mcf7.degree())

in_degrees_mda = dict(G_mda.in_degree())
out_degrees_mda = dict(G_mda.out_degree())
total_degrees_mda = dict(G_mda.degree())

In [ ]:
# density= measure how dense is the graph
density_mcf7 = nx.density(G_mcf7)
print(f"Density MCF7: {density_mcf7}")

density_mda = nx.density(G_mda)
print(f"Density MDA: {density_mda}")

In [ ]:
#connected components= check if the graph is weakly or strongly connected
is_weakly_connected_mcf7 = nx.is_weakly_connected(G_mcf7)
is_strongly_connected_mcf7 = nx.is_strongly_connected(G_mcf7)
print(f"Weakly connected MCF7: {is_weakly_connected_mcf7}")
print(f"Strongly connected MCF7 : {is_strongly_connected_mcf7}")


is_weakly_connected_mda = nx.is_weakly_connected(G_mda)
is_strongly_connected_mda = nx.is_strongly_connected(G_mda)
print(f"Weakly connected MDA: {is_weakly_connected_mda}")
print(f"Strongly connected MDA : {is_strongly_connected_mda}")


In [ ]:
#Shortest path= analyze the shortest paths between nodes
shortest_path_lengths_mcf7 = dict(nx.shortest_path_length(G_mcf7))
shortest_path_lengths_mda = dict(nx.shortest_path_length(G_mda))


# 2. Identify Suppopulations(Communities or Clusters)
Subpopulations in a graph often refer to communities or clusters of nodes that are more densely connected among themselves than with the rest of the graph.:

In [ ]:
#A)Weakly Connected Components:
#For directed graphs, weakly connected components are subgraphs where nodes are connected regardless of edge direction.
weakly_connected_components_mcf7 = list(nx.weakly_connected_components(G_mcf7))
print(f"Number of weakly connected components MCF7: {len(weakly_connected_components_mcf7)}")
weakly_connected_components_mda = list(nx.weakly_connected_components(G_mda))
print(f"Number of weakly connected components MDA: {len(weakly_connected_components_mda)}")



In [ ]:
#B)Strongly Connected Components:
#    Strongly connected components are subgraphs where every node is reachable from every other node, following edge directions.#
strongly_connected_components_mcf7 = list(nx.strongly_connected_components(G_mcf7))
print(f"Number of strongly connected components MCF7: {len(strongly_connected_components_mcf7)}")
strongly_connected_components_mda = list(nx.strongly_connected_components(G_mda))
print(f"Number of strongly connected components MDA: {len(strongly_connected_components_mda)}")

In [ ]:
#D)K-Cores:

#Identify k-cores, which are subgraphs where each node has at least degree k.
G_mcf7.remove_edges_from(nx.selfloop_edges(G_mcf7)) # Remove self-loops from G_mcf7
k_cores_mcf7 = nx.k_core(G_mcf7)
print(f"Number of k-cores in MCF7: {k_cores_mcf7.number_of_nodes()}")

G_mda.remove_edges_from(nx.selfloop_edges(G_mda)) # Remove self-loops from G_mda
k_cores_mda = nx.k_core(G_mda)
print(f"Number of k-cores in MDA: {k_cores_mda.number_of_nodes()}")

In [ ]:
#E) Cliques: Find cliques (fully connected subgraphs).
cliques_mcf7 = list(nx.find_cliques(G_mcf7.to_undirected()))
cliques_mda = list(nx.find_cliques(G_mda.to_undirected()))

# **3 . Analyze Subpopulation Properties**





In [ ]:
#Size of Subpopulations: Check the size of each community or component.
#for i, component in enumerate(weakly_connected_components_mcf7):
 #   print(f"Component {i+1}: Size = {len(component)}")
#Density of Subpopulations: Calculate the density of each community or component
#for i, component in enumerate(weakly_connected_components_mcf7):
 #   subgraph = G_mcf7.subgraph(component)
  #  print(f"Component {i+1}: Density = {nx.density(subgraph)}")
#Centrality Measures: Compute centrality measures (e.g., degree centrality, betweenness centrality) for nodes within each subpopulation.
#for i, component in enumerate(weakly_connected_components_mcf7):
   # subgraph = G_mcf7.subgraph(component)
    #degree_centrality = nx.degree_centrality(subgraph)
    #betweenness_centrality = nx.betweenness_centrality(subgraph)
    #print(f"Component {i+1}: Degree Centrality = {degree_centrality}")
    #print(f"Component {i+1}: Betweenness Centrality = {betweenness_centrality}")
#    Edge Types in Subpopulations:Analyze the distribution of edge types ("positive" or "negative") within each subpopulation
    #for i, component in enumerate(weakly_connected_components_mcf7):
      #  subgraph = G_mcf7.subgraph(component)
       # edge_types = [data['type'] for _, _, data in subgraph.edges(data=True)]
       # print(f"Component {i+1}: Edge Types = {edge_types}")



In [ ]:
#Size of Subpopulations: Check the size of each community or component.
#for i, component in enumerate(weakly_connected_components_mda):
 #   print(f"Component {i+1}: Size = {len(component)}")
#Density of Subpopulations: Calculate the density of each community or component
#for i, component in enumerate(weakly_connected_components_mda):
 #   subgraph = G_mda.subgraph(component)
  #  print(f"Component {i+1}: Density = {nx.density(subgraph)}")
#Centrality Measures: Compute centrality measures (e.g., degree centrality, betweenness centrality) for nodes within each subpopulation.
#for i, component in enumerate(weakly_connected_components_mda):
 #   subgraph = G_mda.subgraph(component)
  #  degree_centrality = nx.degree_centrality(subgraph)
   # betweenness_centrality = nx.betweenness_centrality(subgraph)
    #print(f"Component {i+1}: Degree Centrality = {degree_centrality}")
    #print(f"Component {i+1}: Betweenness Centrality = {betweenness_centrality}")
#    Edge Types in Subpopulations:Analyze the distribution of edge types ("positive" or "negative") within each subpopulation
    #for i, component in enumerate(weakly_connected_components_mda):
    #    subgraph = G_mda.subgraph(component)
     #   edge_types = [data['type'] for _, _, data in subgraph.edges(data=True)]
      #  print(f"Component {i+1}: Edge Types = {edge_types}")



 4: **Identify** **Key** **Nodes** or **Hubs**

Key nodes (hubs) are often critical in biological networks

In [ ]:

#High-Degree Nodes:
#Identify nodes with the highest degree (in-degree, out-degree, or total degree).
top_degree_nodes_mcf7 = sorted(total_degrees_mcf7.items(), key=lambda x: x[1], reverse=True)[:10]
print(f"Top degree nodes: {top_degree_nodes_mcf7}")
#Betweenness Centrality:
#Identify nodes with high betweenness centrality (bridges between communities).
betweenness_mcf7= nx.betweenness_centrality(G_mcf7)
top_betweenness_nodes_mcf7 = sorted(betweenness_mcf7.items(), key=lambda x: x[1], reverse=True)[:10]
print(f"Top betweenness nodes: {top_betweenness_nodes_mcf7}")

#Eigenvector Centrality:
#Identify influential nodes based on their connections to other influential nodes.
eigenvector_centrality_mcf7 = nx.eigenvector_centrality(G_mcf7)
top_eigenvector_nodes_mcf7 = sorted(eigenvector_centrality_mcf7.items(), key=lambda x: x[1], reverse=True)[:10]
print(f"Top eigenvector centrality nodes: {top_eigenvector_nodes_mcf7}")

In [ ]:

#High-Degree Nodes:
#Identify nodes with the highest degree (in-degree, out-degree, or total degree).
top_degree_nodes_mda = sorted(total_degrees_mda.items(), key=lambda x: x[1], reverse=True)[:10]
print(f"Top degree nodes: {top_degree_nodes_mda}")
#Betweenness Centrality:
#Identify nodes with high betweenness centrality (bridges between communities).
betweenness_mda= nx.betweenness_centrality(G_mda)
top_betweenness_nodes_mda = sorted(betweenness_mda.items(), key=lambda x: x[1], reverse=True)[:10]
print(f"Top betweenness nodes: {top_betweenness_nodes_mda}")

#Eigenvector Centrality:
#Identify influential nodes based on their connections to other influential nodes.
eigenvector_centrality_mda = nx.eigenvector_centrality(G_mda)
top_eigenvector_nodes_mda = sorted(eigenvector_centrality_mda.items(), key=lambda x: x[1], reverse=True)[:10]
print(f"Top eigenvector centrality nodes: {top_eigenvector_nodes_mda}")

# 5.Export Results for further analysis

In [ ]:
nx.write_graphml(G_mcf7, "mcf7_graph.graphml")
#expoert subpopulations
with open("subpopulations_mcf7.txt", "w") as f:
    for i, component in enumerate(weakly_connected_components_mcf7):
        f.write(f"Component {i+1}: {component}\n")
nx.write_graphml(G_mda, "mda_graph.graphml")
#expoert subpopulations
with open("subpopulations_mda.txt", "w") as f:
    for i, component in enumerate(weakly_connected_components_mda):
        f.write(f"Component {i+1}: {component}\n")

# Let's install PyGPS (suggested by Ortega) and use it for signal processing:

In [ ]:
pip install pygsp


# **1. Convert NetworkX Graph to PyGSP Graph**

In [ ]:
#I decided to use PyGSP package so few adjustments will be required
import networkx as nx
import pygsp as pg
from networkx import to_scipy_sparse_array

# Extract adjacency matrix from NetworkX graph
# Extract adjacency matrix from NetworkX graph as a sparse array
G_mcf7.remove_edges_from(nx.selfloop_edges(G_mcf7))##### remove the loop here
adjacency_matrix_mcf7 = to_scipy_sparse_array(G_mcf7, weight=" Edge weight")

# Create PyGSP graph
gsp_graph_mcf7= pg.graphs.Graph(adjacency_matrix_mcf7)

# Display basic graph info
print(f"PyGSP Graph MCF7 with {gsp_graph_mcf7.N} nodes and {gsp_graph_mcf7.Ne} edges.")


In [ ]:
## Extract adjacency matrix from NetworkX graph
# Extract adjacency matrix from NetworkX graph as a sparse array
G_mda.remove_edges_from(nx.selfloop_edges(G_mda))##### remove the loop here
adjacency_matrix_mda = to_scipy_sparse_array(G_mda, weight=" Edge weight")

# Create PyGSP graph
gsp_graph_mda= pg.graphs.Graph(adjacency_matrix_mda)

# Display basic graph info
print(f"PyGSP Graph MDA with {gsp_graph_mda.N} nodes and {gsp_graph_mda.Ne} edges.")


# **2.Compute the Laplacian Matrix**


In [ ]:
# Compute Laplacian (default is combinatorial)
gsp_graph_mcf7.compute_laplacian(lap_type="combinatorial")
laplacian_mcf7 = gsp_graph_mcf7.L

print("Laplacian Matrix MCF7:")
print(laplacian_mcf7.toarray())  # Display dense matrix for small graphs


In [ ]:
# Compute Laplacian (default is combinatorial)
gsp_graph_mda.compute_laplacian(lap_type="combinatorial")
laplacian_mda = gsp_graph_mda.L

print("Laplacian Matrix Neg:")
print(laplacian_mda.toarray())  # Display dense matrix for small graphs


# 3. Assign Graph Signal
Assign RNa-seq data as the graph signal

In [ ]:
import pandas as pd

# Load RNA-seq data
rna_seq_data = pd.read_csv("/content/drive/MyDrive/Essex_MSc_AI_24-25/MSc_Project_24/Data_to_work_with/GRAND_datasets/significant_genes_cleaned.tsv", sep='\t')  # Replace with your RNA-seq file
# Columns: gene, expression


# Extract nodes for G_mcf7 and G_mda
mcf7_genes = set(G_mcf7.nodes)
mda_genes = set(G_mda.nodes)
all_genes_in_graph = mcf7_genes.union(mda_genes)

# Filter RNA-seq data for genes in G_mcf7 and G_mda
mcf7_rna_seq = rna_seq_data[rna_seq_data['gene_name'].isin(mcf7_genes)]
mda_rna_seq = rna_seq_data[rna_seq_data['gene_name'].isin(mda_genes)]

# Create dictionaries for each graph
# Assign expression = 0 for genes not present in the RNA-seq data
expression_dict_mcf7 = {gene: 0 for gene in mcf7_genes}  # Initialize all genes with expression = 0
expression_dict_mcf7.update(zip(mcf7_rna_seq['gene_name'], mcf7_rna_seq['MCF7']))  # Update with RNA-seq values

expression_dict_mda = {gene: 0 for gene in mda_genes}  # Initialize all genes with expression = 0
expression_dict_mda.update(zip(mda_rna_seq['gene_name'], mda_rna_seq['MDA-MB231']))  # Update with RNA-seq values

# Assign expression values to nodes in G_mcf7 and G_mda
nx.set_node_attributes(G_mcf7, expression_dict_mcf7, name='expression')
nx.set_node_attributes(G_mda, expression_dict_mda, name='expression')

# Verify attributes are set correctly
print("G_mcf7 node attributes:")
for node, data in G_mcf7.nodes(data=True):
    print(node, data)

print("\nG_mda node attributes:")
for node, data in G_mda.nodes(data=True):
    print(node, data)

In [ ]:
import numpy as np

def identify_highly_expressed_genes(graph, threshold):
    """
    Identify genes with expression levels above a given threshold.
    """
    return {
        node: data['expression']
        for node, data in graph.nodes(data=True)
        if data['expression'] > threshold
    }

# Set a threshold for high expression (e.g., top 10% of expression values)
threshold_mcf7 = np.percentile(list(expression_dict_mcf7.values()), 90)
threshold_mda = np.percentile(list(expression_dict_mda.values()), 90)

# Find highly expressed genes in G_mcf7 and G_mda
highly_expressed_mcf7 = identify_highly_expressed_genes(G_mcf7, threshold_mcf7)
highly_expressed_mda = identify_highly_expressed_genes(G_mda, threshold_mda)

print("Highly expressed genes in G_mcf7:")
print(highly_expressed_mcf7)

print("\nHighly expressed genes in G_mda:")
print(highly_expressed_mda)

def compare_expression_profiles(graph1, graph2):
    """
    Compare expression profiles between two graphs for common genes,
    excluding genes with zero expression in both graphs.
    """
    common_genes = set(graph1.nodes).intersection(set(graph2.nodes))
    return {
        gene: (graph1.nodes[gene]['expression'], graph2.nodes[gene]['expression'])
        for gene in common_genes
        if not (graph1.nodes[gene]['expression'] == 0 and graph2.nodes[gene]['expression'] == 0)
    }

# Compare expression profiles between G_mcf7 and G_mda
expression_comparison = compare_expression_profiles(G_mcf7, G_mda)

print("\nExpression comparison for common genes:")
for gene, (exp_mcf7, exp_mda) in expression_comparison.items():
    print(f"{gene}: MCF7 = {exp_mcf7}, MDA-MB231 = {exp_mda}")

def differential_expression_analysis(graph1, graph2, fold_change_threshold=2.0):
    """
    Perform differential expression analysis between two graphs.
    Genes are considered differentially expressed if the fold change exceeds the threshold,
    excluding genes with zero expression in both cell lines and avoiding infinite fold changes.
    """
    common_genes = set(graph1.nodes).intersection(set(graph2.nodes))
    differentially_expressed_genes = {}

    for gene in common_genes:
        exp_mcf7 = graph1.nodes[gene]['expression']
        exp_mda = graph2.nodes[gene]['expression']

        # Exclude genes with zero expression in both cell lines
        if exp_mcf7 == 0 and exp_mda == 0:
            continue

        # Compute fold change while avoiding division by zero
        if exp_mda == 0:
            continue  # Exclude cases leading to infinite fold change

        fold_change = exp_mcf7 / exp_mda

        # Check if fold change exceeds the threshold
        if abs(fold_change) > fold_change_threshold:
            differentially_expressed_genes[gene] = (exp_mcf7, exp_mda, fold_change)

    return differentially_expressed_genes

# Perform differential expression analysis
differentially_expressed_genes = differential_expression_analysis(G_mcf7, G_mda, fold_change_threshold=2)

print("\nDifferentially expressed genes (fold change > 2.0):")
for gene, (exp_mcf7, exp_mda, fold_change) in differentially_expressed_genes.items():
    print(f"{gene}: MCF7 = {exp_mcf7}, MDA-MB231 = {exp_mda}, Fold Change = {fold_change:.2f}")


# **5. Analyze Signal Variation**

In [ ]:
# ... previous code ...

# After setting node attributes with expression values
# Extract node expression values for G_mcf7
signal_mcf7 = np.array([data['expression'] for node, data in G_mcf7.nodes(data=True)])

# Extract node expression values for G_mda
signal_mda = np.array([data['expression'] for node, data in G_mda.nodes(data=True)])

# Signal variation for MCF7
signal_variation_mcf7 = signal_mcf7.T @ laplacian_mcf7 @ signal_mcf7 # Changed laplacian to laplacian_mcf7
print(f"\nSignal Variation (MCF7): {signal_variation_mcf7:.2e}")

# Signal variation for MDA
signal_variation_mda = signal_mda.T @ laplacian_mda @ signal_mda # Changed laplacian to laplacian_mda
print(f"\nSignal Variation (MDA): {signal_variation_mda:.2e}")

# 6.**Optional: Visualize Signal Distribution**

In [ ]:
import matplotlib.pyplot as plt

# Plot RNA-seq signals for positive graph
plt.figure(figsize=(10, 6))
plt.hist(signal_mcf7, bins=30, alpha=0.7, label="MCF7 graph", color='blue')
plt.hist(signal_mda, bins=30, alpha=0.7, label="MDA graph", color='orange')
plt.xlabel("RNA-seq Expression Level")
plt.ylabel("Frequency")
plt.title("Distribution of RNA-seq Expression Levels")
plt.legend()
plt.show()


# Next Steps
Clustering:
Perform spectral clustering using the eigenvalues and eigenvectors of the Laplacians.

Filtering:
Apply graph filters to smooth or enhance RNA-seq signals on the graphs.

Comparative Analysis:
Compare the results from the positive and negative graphs to identify key genes or regulatory patterns.

# Spectral Clustering
We’ll now perform spectral clustering using the eigenvalues and eigenvectors of the Laplacian matrices for the positive and negative graphs. This helps group nodes (genes) into clusters based on their connectivity patterns and RNA-seq signal variations.

Steps for Spectral Clustering:

A) Compute Eigenvalues and Eigenvectors

Use PyGSP to extract the Fourier basis (eigenvectors of the Laplacian) and corresponding eigenvalues.

B) Cluster Nodes

Use the eigenvectors corresponding to the smallest non-zero eigenvalues for clustering (spectral embedding).

C) Visualize Clustering

Optionally visualize clusters for insights.


##1. Split the Graph into Positive and Negative Subgraphs
###Since your graph contains both positive and negative edge weights, a common approach is to split the graph into two separate subgraphs:
### Positive Subgraph : Contains only edges with positive weights (activation).
### Negative Subgraph : Contains only edges with negative weights (suppression).
###This allows you to analyze the activation and suppression networks separately.

In [ ]:
# Create positive and negative subgraphs
G_positive_MCF7 = nx.DiGraph()
G_negative_MCF7= nx.DiGraph()

for u, v, data in G_mcf7.edges(data=True):
    weight = data.get("weight", 0)
    if weight > 0:
        G_positive_MCF7.add_edge(u, v, weight=weight)
    elif weight < 0:
        G_negative_MCF7.add_edge(u, v, weight=abs(weight))  # Use absolute value for negative weights

print(f"Positive Subgraph: {G_positive_MCF7.number_of_edges()} edges")
print(f"Negative Subgraph: {G_negative_MCF7.number_of_edges()} edges")

In [ ]:
# Create positive and negative subgraphs
G_positive_MDA = nx.DiGraph()
G_negative_MDA= nx.DiGraph()

for u, v, data in G_mda.edges(data=True):
    weight = data.get("weight", 0)
    if weight > 0:
        G_positive_MDA.add_edge(u, v, weight=weight)
    elif weight < 0:
        G_negative_MDA.add_edge(u, v, weight=abs(weight))  # Use absolute value for negative weights

print(f"Positive Subgraph: {G_positive_MDA.number_of_edges()} edges")
print(f"Negative Subgraph: {G_negative_MDA.number_of_edges()} edges")

# 2. Analyze Each Subgraph Separately
Once you’ve split the graph, you can perform spectral clustering, signal processing, or other analyses on each subgraph independently.

Steps:
Compute Laplacian Matrices :

*   Compute the Laplacian for the positive and negative subgraphs.
*  For the negative subgraph, you may want to reverse the sign of the weights to treat suppression as a form of "activation" during analysis.

Perform Spectral Clustering :

*   Use the eigenvectors of the Laplacian matrices to cluster nodes in each subgraph.

Combine Results :

*  After analyzing both subgraphs, combine the results to interpret the overall regulatory network.

#3. Handle Disconnected Components
Both the positive and negative subgraphs may contain disconnected components. Ensure that you handle these components separately during Fourier basis computation and clustering.


In [ ]:
import networkx as nx
import pygsp as pg
from sklearn.cluster import KMeans
import numpy as np

def process_subgraph(graph_nx, cell_line_name):
    print(f"{cell_line_name} graph is not connected. Handling each component separately.")

    # Identify connected components
    if graph_nx.is_directed():
        connected_components = list(nx.weakly_connected_components(graph_nx))
    else:
        connected_components = list(nx.connected_components(graph_nx))

    all_clusters = []
    for i, component in enumerate(connected_components):
        print(f"Processing component {i+1} with {len(component)} nodes...")

        # Create a subgraph for the component
        subgraph_nx = graph_nx.subgraph(component)
        adjacency_matrix = nx.to_scipy_sparse_array(subgraph_nx, weight="weight")
        subgraph_pygsp = pg.graphs.Graph(adjacency_matrix)

        try:
            # Compute Laplacian and Fourier basis for the subgraph
            subgraph_pygsp.compute_laplacian(lap_type="combinatorial")
            subgraph_pygsp.compute_fourier_basis(recompute=True)

            # Clamp smallest eigenvalue to 0 if it's very close
            if abs(subgraph_pygsp.e[0]) < 1e-10:
                subgraph_pygsp.e[0] = 0

            # Perform clustering on the subgraph
            num_clusters = 3  # Number of clusters
            clusters_subgraph = KMeans(n_clusters=num_clusters).fit_predict(
                subgraph_pygsp.U[:, 1:num_clusters+1]
            )

            # Map cluster labels back to original node indices
            cluster_dict = dict(zip(subgraph_nx.nodes, clusters_subgraph))
            all_clusters.append(cluster_dict)

            print(f"Fourier basis and clustering computed for component {i+1}.")
        except Exception as e:
            print(f"Failed to compute Fourier basis for component {i+1}: {e}")

    # Combine cluster assignments for all components
    combined_clusters = {}
    for cluster_dict in all_clusters:
        combined_clusters.update(cluster_dict)

    return combined_clusters

# Process MCF7 positive and negative subgraphs
clusters_positive_mcf7 = process_subgraph(G_positive_MCF7, "MCF7 Positive")
clusters_negative_mcf7 = process_subgraph(G_negative_MCF7, "MCF7 Negative")

# Process MDA positive and negative subgraphs
clusters_positive_mda = process_subgraph(G_positive_MDA, "MDA Positive")
clusters_negative_mda = process_subgraph(G_negative_MDA, "MDA Negative")

# Assign cluster labels to nodes in the NetworkX graphs
nx.set_node_attributes(G_positive_MCF7, clusters_positive_mcf7, "cluster")
nx.set_node_attributes(G_negative_MCF7, clusters_negative_mcf7, "cluster")
nx.set_node_attributes(G_positive_MDA, clusters_positive_mda, "cluster")
nx.set_node_attributes(G_negative_MDA, clusters_negative_mda, "cluster")

print("Cluster assignments completed.")

In [ ]:
import matplotlib.pyplot as plt

def visualize_graph(graph, title):
    pos = nx.spring_layout(graph)
    plt.figure(figsize=(12, 8))
    nx.draw_networkx_nodes(graph, pos, node_size=50, node_color=[data['cluster'] for _, data in graph.nodes(data=True)], cmap=plt.cm.Set3)
    nx.draw_networkx_edges(graph, pos, alpha=0.5)
    plt.title(title)
    plt.show()

#visualize_graph(G_positive_MCF7, "Clustering of MCF7 Positive Graph Nodes")
#visualize_graph(G_negative_MCF7, "Clustering of MCF7 Negative Graph Nodes")
#visualize_graph(G_positive_MDA, "Clustering of MDA Positive Graph Nodes")
#visualize_graph(G_negative_MDA, "Clustering of MDA Negative Graph Nodes")


In [ ]:
# Compute Fourier basis (eigenvalues and eigenvectors) for both graphs
gsp_graph_mcf7.compute_fourier_basis(recompute=True)
gsp_graph_mda.compute_fourier_basis(recompute=True)

# Clamp smallest eigenvalue to 0 if it's very close
if abs(gsp_graph_mcf7.e[0]) < 1e-10:
    gsp_graph_mcf7.e[0] = 0
if abs(gsp_graph_mda.e[0]) < 1e-10:
    gsp_graph_mda.e[0] = 0

# Extract eigenvalues and eigenvectors
eigvals_mcf7 = gsp_graph_mcf7.e  # Eigenvalues
eigvecs_mcf7 = gsp_graph_mcf7.U  # Eigenvectors

eigvals_mda = gsp_graph_mda.e  # Eigenvalues
eigvecs_mda = gsp_graph_mda.U  # Eigenvectors

# Display smallest eigenvalues (skip 0 for connected components)
print("\nEigenvalues (MCF7 Graph):", eigvals_mcf7[:10])  # First 10 for brevity
print("\nEigenvalues (MDA Graph):", eigvals_mda[:10])  # First 10 for brevity

In [ ]:
import matplotlib.pyplot as plt

# Get the graph signals
signal_mcf7 = nx.get_node_attributes(G_mcf7, 'expression')
signal_mda = nx.get_node_attributes(G_mda, 'expression')
signal_mcf7 = list(signal_mcf7.values())
signal_mda = list(signal_mda.values())

# Plot RNA-seq signals for positive graph
plt.figure(figsize=(10, 6))
plt.hist(signal_mcf7, bins=30, alpha=0.7, label="MCF7 graph", color='blue')
plt.hist(signal_mda, bins=30, alpha=0.7, label="MDA graph", color='orange')
plt.xlabel("RNA-seq Expression Level")
plt.ylabel("Frequency")
plt.title("Distribution of RNA-seq Expression Levels")
plt.legend()
plt.show()

In [ ]:
#Perform Spectral Clustering
#Use the eigenvectors associated with the smallest non-zero eigenvalues to cluster the nodes (genes).

from sklearn.cluster import KMeans

# Number of clusters
num_clusters = 3

# Use first few eigenvectors (excluding the first constant eigenvector)
clusters_mcf7 = KMeans(n_clusters=num_clusters).fit_predict(eigvecs_mcf7[:, 1:num_clusters+1])
clusters_mda = KMeans(n_clusters=num_clusters).fit_predict(eigvecs_mda[:, 1:num_clusters+1])

print("\nClusters (Positive Graph):", clusters_mcf7[:10])  # First 10 clusters for brevity
print("\nClusters (Negative Graph):", clusters_mda[:10])  # First 10 clusters for brevity)


In [ ]:
#Assign Clusters to Nodes
#Assign the cluster labels to nodes in the NetworkX graph for further exploration.

# Assign cluster labels as node attributes
nx.set_node_attributes(G_mcf7, dict(zip(G_mcf7.nodes, clusters_mcf7)), "cluster")
nx.set_node_attributes(G_mda, dict(zip(G_mda.nodes, clusters_mda)), "cluster")

# Verify node clusters
print("\nG_positive Clusters:")
for node, data in list(G_mcf7.nodes(data=True))[:5]:  # First 5 for brevity
    print(node, data)

print("\nG_negative Clusters:")
for node, data in list(G_mda.nodes(data=True))[:5]:  # First 5 for brevity
    print(node, data)




In [ ]:
#Visualize Clustering Results
#(Optional) You can visualize the clustering using a library like Matplotlib.

import matplotlib.pyplot as plt
import networkx as nx

# Visualize positive graph clustering
pos = nx.spring_layout(G_mcf7)  # Spring layout for visualization
plt.figure(figsize=(12, 8))
nx.draw_networkx_nodes(G_mcf7, pos, node_size=50, node_color=clusters_mcf7, cmap=plt.cm.Set3)
nx.draw_networkx_edges(G_mcf7, pos, alpha=0.3)
plt.title("Clustering of MCF7 Graph Nodes")
plt.show()

# Visualize negative graph clustering
pos = nx.spring_layout(G_mda)  # Spring layout for visualization
plt.figure(figsize=(12, 8))
nx.draw_networkx_nodes(G_mda, pos, node_size=50, node_color=clusters_mda, cmap=plt.cm.Set3)
nx.draw_networkx_edges(G_mda, pos, alpha=0.3)
plt.title("Clustering of Negative Graph Nodes")
plt.show()


# Lets try to identify the positive graph with stronger interaction hub depicted with thicker edges

# 1. Identify the Hub Node
# The hub node is likely the node with the highest degree or the strongest total edge weight. You can calculate these metrics:

# Degree Centrality (Number of Connections)

In [ ]:
# Get degree centrality for G_positive
degree_centrality = nx.degree_centrality(G_mcf7)

# Sort nodes by degree centrality
sorted_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)

# Display the top nodes
print("Top Nodes by Degree Centrality (Positive Graph):")
for node, centrality in sorted_degree[:5]:  # Top 5 nodes
    print(f"Node: {node}, Degree Centrality: {centrality}")


In [ ]:
# Compute strength (sum of edge weights) for each node
strength = {node: sum(data["weight"] for _, _, data in G_positive.edges(node, data=True)) for node in G_positive.nodes()}

# Sort nodes by strength
sorted_strength = sorted(strength.items(), key=lambda x: x[1], reverse=True)

# Display the top nodes
print("\nTop Nodes by Strength (Edge Weights, Positive Graph):")
for node, weight in sorted_strength[:5]:  # Top 5 nodes
    print(f"Node: {node}, Strength (Sum of Weights): {weight}")


# **2. Identify Genes and TFs Connected to the Hub**
# Once you identify the hub node, find all nodes connected to it and analyze their edge weights:

In [ ]:
# Get the top hub node (by strength or degree centrality)
top_hub_node = sorted_strength[0][0]  # Replace with your preferred metric

# Get neighbors (connected nodes) of the hub
neighbors = G_positive[top_hub_node]

print(f"\nNeighbors of Hub Node ({top_hub_node}):")
for neighbor, edge_data in neighbors.items():
    print(f"Connected Node: {neighbor}, Edge Weight: {edge_data['weight']}")


In [ ]:
top_hub_node = sorted_degree[0][0]  # Use degree centrality to select the hub
# Get neighbors (connected nodes) of the hub
neighbors = G_positive[top_hub_node]

print(f"\nNeighbors of Hub Node ({top_hub_node}):")
for neighbor, edge_data in neighbors.items():
    print(f"Connected Node: {neighbor}, Edge Weight: {edge_data['weight']}")

# **3. Visualize the Hub and Its Connections**
# You can highlight the hub and its neighbors in a graph visualization:

In [ ]:
import matplotlib.pyplot as plt

# Get the subgraph for the hub and its neighbors
hub_subgraph = G_positive.subgraph([top_hub_node] + list(neighbors.keys()))

# Visualize the subgraph
pos = nx.spring_layout(hub_subgraph)  # Layout for visualization
plt.figure(figsize=(8, 6))
nx.draw_networkx_nodes(hub_subgraph, pos, node_size=700, node_color="lightblue")
nx.draw_networkx_edges(hub_subgraph, pos, width=[d["weight"] for (_, _, d) in hub_subgraph.edges(data=True)])
nx.draw_networkx_labels(hub_subgraph, pos, font_size=10)
plt.title(f"Hub Node ({top_hub_node}) and Its Connections")
plt.show()


# **4. Export Hub Connections for Further Analysis**
# If you want to analyze this data outside Python, export the hub’s connections to a file:

In [ ]:
import pandas as pd

# Create a DataFrame for the hub's connections
hub_connections = pd.DataFrame({
    "TF": [top_hub_node] * len(neighbors),
    "Gene": list(neighbors.keys()),
    "Edge Weight": [data["weight"] for _, data in neighbors.items()]
})

# Save to CSV
hub_connections.to_csv("hub_connections_positive.csv", index=False)
print("\nHub connections exported to 'hub_connections_positive.csv'")
